In [14]:
#first import all the required library,let's first use breast wine dataset
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from sklearn.ensemble import RandomForestClassifier
from IPython import display
import LeeExperimentToolkit as lee
from matplotlib import pyplot as plt
from sklearn.datasets import load_wine
import random
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier

first thing is to get a initial dataset to fit in active learning

In [2]:
X, y = load_wine(return_X_y=True,as_frame=True)

use Leetoolkit library we can get a dataset with onehot representation oof feature and its associated cv means

In [3]:
X_initial, y_initial = lee.PerformanceHandler.train_dataset_with_random_features(X,y,RandomForestClassifier(),2000)#get a dataset with size of 2000

In [4]:
#see the information of two dataset
X_initial.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,1,1,0,1,1,1,1,1,1,1,1,1
1,1,1,0,1,1,1,1,1,1,1,1,1,1
2,1,1,0,1,0,1,0,1,1,1,0,1,1
3,0,0,1,1,1,0,1,1,1,1,1,0,1
4,0,1,0,0,0,1,1,0,0,0,1,1,0


In [18]:
y_initial.value_counts()

8    828
7    555
6    157
5    104
4     63
3     53
2     30
0     16
1     15
Name: 14, dtype: int64

now we split this dataset into trainning and testing dataset

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X_initial,y_initial,test_size=0.5)

In [9]:
#see information of test set, we will test all iteration by this test size so we keep it bigger
len(y_test)

911

In [10]:
def Change_float_into_classes(num):
    if num> 0.95:
        return 8
    elif num > 0.9:
        return 7
    elif num > 0.85:
        return 6
    elif num > 0.8:
        return 5
    elif num > 0.75:
        return 4
    elif num > 0.7:
        return 3
    elif num > 0.65:
        return 2
    elif num > 0.6:
        return 1
    else:
        return 0

now let's create the pool, the pool should be different combination of feature compare to the initial set, we can use lee library to do this

In [15]:
X_pool = lee.PerformanceHandler.pool_generator(X_initial,2000)

In [16]:
X_pool = np.array(X_pool)

In [17]:
N_QUERIES = 100
X_train =np.array(X_train)
y_train = np.array(y_train)
learner = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train,y_training=y_train)
unqueried_score = [learner.score(X_test, y_test)]
performance_history = [unqueried_score]

# Allow our model to query our unlabeled dataset for the most
# informative points according to our query strategy (uncertainty sampling).
for index in range(N_QUERIES):
  query_index, query_instance = learner.query(X_pool)

  # Teach our ActiveLearner model the record it has requested.
  X_Iterate = X.iloc[:,query_instance[0].tolist()]
  y_new = np.array([Change_float_into_classes(cross_val_score(estimator=RandomForestClassifier(),X = X_Iterate,y = y,cv=4).mean())], dtype=int)
  X_1, y_1 = X_pool[query_index].reshape(1, -1), y_new.reshape(1,)
  learner.teach(X=X_1, y=y_1)
  # Remove the queried instance from the unlabeled pool.
  X_pool = np.delete(X_pool, query_index, axis=0)
  # Calculate and report our model's accuracy.
  model_accuracy = learner.score(X_test, y_test)
  print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))

  # Save our model's performance for plotting.
  performance_history.append(model_accuracy)

Accuracy after query 1: 0.7344
Accuracy after query 2: 0.7311
Accuracy after query 3: 0.7387
Accuracy after query 4: 0.7398
Accuracy after query 5: 0.7377
Accuracy after query 6: 0.7398
Accuracy after query 7: 0.7431
Accuracy after query 8: 0.7420
Accuracy after query 9: 0.7475
Accuracy after query 10: 0.7398
Accuracy after query 11: 0.7355
Accuracy after query 12: 0.7420
Accuracy after query 13: 0.7453
Accuracy after query 14: 0.7387
Accuracy after query 15: 0.7245
Accuracy after query 16: 0.7267
Accuracy after query 17: 0.7387
Accuracy after query 18: 0.7377
Accuracy after query 19: 0.7256
Accuracy after query 20: 0.7355
Accuracy after query 21: 0.7300
Accuracy after query 22: 0.7300
Accuracy after query 23: 0.7267
Accuracy after query 24: 0.7245
Accuracy after query 25: 0.7289
Accuracy after query 26: 0.7322
Accuracy after query 27: 0.7234
Accuracy after query 28: 0.7344
Accuracy after query 29: 0.7256
Accuracy after query 30: 0.7168
Accuracy after query 31: 0.7157
Accuracy after qu

In [19]:
direct_trainning_score = cross_val_score(RandomForestClassifier(),X_initial,y_initial,cv=4)

In [24]:
direct_trainning_score

array([0.76754386, 0.78901099, 0.77142857, 0.78681319])

In [25]:
direct_trainning_score_decision_tree = cross_val_score(DecisionTreeClassifier(),X_initial,y_initial,cv=4)

In [26]:
direct_trainning_score_decision_tree

array([0.74342105, 0.75164835, 0.76923077, 0.76703297])

a funny thing came out, the active learning strategies seems to wrose the accuracy

let's see whether the accuracy is already the best randomforrst can get, we will create another even bigger training set

In [27]:
X_4000, y_4000 = lee.PerformanceHandler.train_dataset_with_random_features(X,y,RandomForestClassifier(),4000)#get a dataset with size of 4000

In [28]:
direct_trainning_score_4000 = cross_val_score(RandomForestClassifier(),X_4000,y_4000,cv=4)

In [29]:
direct_trainning_score_4000

array([0.84252874, 0.83889528, 0.83544304, 0.82508631])

In [30]:
direct_trainning_score_decision_tree_4000 = cross_val_score(DecisionTreeClassifier(),X_4000,y_4000,cv=4)

In [31]:
direct_trainning_score_decision_tree_4000

array([0.82873563, 0.81933257, 0.80552359, 0.80322209])

so basically we are not reach the performance limit of randomforrest, and the problem belongs to AL
but still let's first reduce the dataset, and see what will happen if we have a rather small dataset

In [32]:
X_initial_400, y_initial_400 = lee.PerformanceHandler.train_dataset_with_random_features(X,y,RandomForestClassifier(),400)#get a dataset with size of 400
X_train,X_test,y_train,y_test = train_test_split(X_initial_400,y_initial_400,test_size=0.5)
X_pool = lee.PerformanceHandler.pool_generator(X_initial,2000)
X_pool = np.array(X_pool)
N_QUERIES = 100
X_train =np.array(X_train)
y_train = np.array(y_train)
learner = ActiveLearner(estimator=DecisionTreeClassifier(),X_training=X_train,y_training=y_train)
unqueried_score = [learner.score(X_test, y_test)]
performance_history = [unqueried_score]

# Allow our model to query our unlabeled dataset for the most
# informative points according to our query strategy (uncertainty sampling).
for index in range(N_QUERIES):
  query_index, query_instance = learner.query(X_pool)

  # Teach our ActiveLearner model the record it has requested.
  X_Iterate = X.iloc[:,query_instance[0].tolist()]
  y_new = np.array([Change_float_into_classes(cross_val_score(estimator=RandomForestClassifier(),X = X_Iterate,y = y,cv=5,n_jobs=-1).mean())], dtype=int)
  X_1, y_1 = X_pool[query_index].reshape(1, -1), y_new.reshape(1,)
  learner.teach(X=X_1, y=y_1)
  # Remove the queried instance from the unlabeled pool.
  X_pool = np.delete(X_pool, query_index, axis=0)
  # Calculate and report our model's accuracy.
  model_accuracy = learner.score(X_test, y_test)
  print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))

  # Save our model's performance for plotting.
  performance_history.append(model_accuracy)

Accuracy after query 1: 0.5684
Accuracy after query 2: 0.5632
Accuracy after query 3: 0.5632
Accuracy after query 4: 0.5421
Accuracy after query 5: 0.5316
Accuracy after query 6: 0.5684
Accuracy after query 7: 0.5684
Accuracy after query 8: 0.5737
Accuracy after query 9: 0.5632
Accuracy after query 10: 0.5526
Accuracy after query 11: 0.5105
Accuracy after query 12: 0.4947
Accuracy after query 13: 0.5368
Accuracy after query 14: 0.4895
Accuracy after query 15: 0.5632
Accuracy after query 16: 0.5526
Accuracy after query 17: 0.5526
Accuracy after query 18: 0.5474
Accuracy after query 19: 0.5263
Accuracy after query 20: 0.5158
Accuracy after query 21: 0.5158
Accuracy after query 22: 0.5579
Accuracy after query 23: 0.5158
Accuracy after query 24: 0.4789
Accuracy after query 25: 0.5105
Accuracy after query 26: 0.5053
Accuracy after query 27: 0.5263
Accuracy after query 28: 0.5263
Accuracy after query 29: 0.4895
Accuracy after query 30: 0.4895
Accuracy after query 31: 0.4947
Accuracy after qu